In [1]:
import pandas as pd
import numpy as np

In [2]:
d1 = pd.read_csv("reviews_data1.csv")

In [3]:
reviews = d1['Reviews']
sentiment = d1['Recommended']

In [4]:
change_format_sentiment = []
for sent in sentiment:
    if sent == 'yes':
        change_format_sentiment.append("Pos")
    else:
        change_format_sentiment.append("Neg")

In [5]:
df_1 = pd.DataFrame(columns = ["Review", "Sentiment"])

for i in range(len(reviews)):
    row = pd.Series({'Review' : reviews[i], 'Sentiment' : change_format_sentiment[i]})
    df_1 = pd.concat([df_1, row.to_frame().T], ignore_index = True)

In [6]:
vocab = df_1['Review']

In [7]:
#import glove embeddings 
from tqdm import tqdm
embedding_vector = {}
f = open('glove.6B.200d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400000it [00:07, 55702.02it/s]


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
token = Tokenizer()
token.fit_on_texts(vocab)

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_1, test_size=0.1, random_state=42)

In [12]:
x_train = list(train['Review'])
y_train = train['Sentiment']

In [13]:
vocab_size = len(token.word_index)+1
print(vocab_size)

14504


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

seq = token.texts_to_sequences(x_train)
pad_seq = pad_sequences(seq, maxlen=1100, padding = 'post')

In [15]:
from tqdm import tqdm
embedding_matrix = np.zeros((vocab_size,200))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|████████████████████████████████████| 14503/14503 [00:00<00:00, 332715.22it/s]


In [16]:
embedding_matrix.shape

(14504, 200)

In [19]:
#convert positive-negative to 1-0
sentiment = {
    "Pos" : 0,
    "Neg" : 1
}

y_filtered_converted = []
for sent in y_train:
    y_filtered_converted.append(sentiment[sent])

In [20]:
y_filtered_converted = np.int64(y_filtered_converted)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional, Conv1D, Flatten

In [26]:
from tensorflow import keras

In [28]:
model_4 = Sequential()
model_4.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_4.add(Bidirectional(LSTM(256, return_sequences = True, kernel_regularizer = keras.regularizers.l1())))
model_4.add(Bidirectional(LSTM(128, return_sequences = True, kernel_regularizer = keras.regularizers.l1())))
model_4.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_4.add(Flatten())


model_4.add(Dense(1,activation = 'sigmoid'))

model_4.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [29]:
history_4 = model_4.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=128, 
                    verbose=1, 
                    epochs=8,
                    validation_split=0.2)

Epoch 1/8
20/20 [==============================] - 223s 11s/step - loss: 269.5666 - accuracy: 0.5484 - val_loss: 189.2313 - val_accuracy: 0.5640
Epoch 2/8
20/20 [==============================] - 245s 12s/step - loss: 135.7446 - accuracy: 0.6368 - val_loss: 81.1369 - val_accuracy: 0.6370
Epoch 3/8
20/20 [==============================] - 243s 12s/step - loss: 49.6504 - accuracy: 0.6668 - val_loss: 21.4184 - val_accuracy: 0.6872
Epoch 4/8
20/20 [==============================] - 266s 13s/step - loss: 11.7348 - accuracy: 0.6486 - val_loss: 6.4269 - val_accuracy: 0.5673
Epoch 5/8
20/20 [==============================] - 259s 13s/step - loss: 4.2686 - accuracy: 0.5890 - val_loss: 2.9318 - val_accuracy: 0.5818
Epoch 6/8
20/20 [==============================] - 237s 12s/step - loss: 2.4830 - accuracy: 0.6015 - val_loss: 2.1967 - val_accuracy: 0.5624
Epoch 7/8
20/20 [==============================] - 248s 12s/step - loss: 2.0593 - accuracy: 0.5934 - val_loss: 1.9887 - val_accuracy: 0.5624
Epo

In [39]:
model_5 = Sequential()
model_5.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_5.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_5.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_5.add(Conv1D(128, 3, activation='tanh', padding = 'same'))
model_5.add(Bidirectional(LSTM(256, return_sequences = True, kernel_regularizer = keras.regularizers.l1())))
model_5.add(Bidirectional(LSTM(128, return_sequences = True)))
model_5.add(Bidirectional(LSTM(128)))


model_5.add(Dense(1,activation = 'sigmoid'))

model_5.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [40]:
model_5.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1100, 200)         2900800   
                                                                 
 conv1d_9 (Conv1D)           (None, 1100, 256)         153856    
                                                                 
 conv1d_10 (Conv1D)          (None, 1100, 256)         196864    
                                                                 
 conv1d_11 (Conv1D)          (None, 1100, 128)         98432     
                                                                 
 bidirectional_15 (Bidirecti  (None, 1100, 512)        788480    
 onal)                                                           
                                                                 
 bidirectional_16 (Bidirecti  (None, 1100, 256)        656384    
 onal)                                                

In [41]:
history_5 = model_5.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=128, 
                    verbose=1, 
                    epochs=8,
                    validation_split=0.2)

Epoch 1/8
20/20 [==============================] - 308s 15s/step - loss: 73.5719 - accuracy: 0.5752 - val_loss: 50.8014 - val_accuracy: 0.6143
Epoch 2/8
20/20 [==============================] - 333s 17s/step - loss: 35.8933 - accuracy: 0.6490 - val_loss: 20.8761 - val_accuracy: 0.6175
Epoch 3/8
20/20 [==============================] - 345s 17s/step - loss: 12.3610 - accuracy: 0.6927 - val_loss: 5.0307 - val_accuracy: 0.6370
Epoch 4/8
20/20 [==============================] - 338s 17s/step - loss: 2.9396 - accuracy: 0.7134 - val_loss: 1.9505 - val_accuracy: 0.6596
Epoch 5/8
 4/20 [=====>........................] - ETA: 4:14 - loss: 1.7783 - accuracy: 0.7051

KeyboardInterrupt: 

In [42]:
model_6 = Sequential()
model_6.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_6.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_6.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_6.add(Conv1D(128, 3, activation='tanh', padding = 'same'))
model_6.add(Bidirectional(LSTM(256, return_sequences = True, kernel_regularizer = keras.regularizers.l1())))
model_6.add(Bidirectional(LSTM(256, return_sequences = True)))
model_6.add(Bidirectional(LSTM(128, return_sequences = True)))
model_6.add(Bidirectional(LSTM(128)))


model_6.add(Dense(1,activation = 'sigmoid'))

model_6.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [43]:
history_6 = model_6.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=128, 
                    verbose=1, 
                    epochs=15,
                    validation_split=0.2)

Epoch 1/15
20/20 [==============================] - 734s 37s/step - loss: 73.7427 - accuracy: 0.5630 - val_loss: 50.9526 - val_accuracy: 0.5673
Epoch 2/15
20/20 [==============================] - 746s 37s/step - loss: 36.0210 - accuracy: 0.6380 - val_loss: 20.8909 - val_accuracy: 0.6791
Epoch 3/15
20/20 [==============================] - 608s 30s/step - loss: 12.3801 - accuracy: 0.7065 - val_loss: 5.0970 - val_accuracy: 0.6418
Epoch 4/15
20/20 [==============================] - 592s 30s/step - loss: 2.9989 - accuracy: 0.6623 - val_loss: 1.8152 - val_accuracy: 0.7164
Epoch 5/15
20/20 [==============================] - 595s 30s/step - loss: 1.3430 - accuracy: 0.7438 - val_loss: 1.6192 - val_accuracy: 0.5835
Epoch 6/15
20/20 [==============================] - 586s 29s/step - loss: 1.2513 - accuracy: 0.7227 - val_loss: 1.3104 - val_accuracy: 0.4360
Epoch 7/15
20/20 [==============================] - 577s 29s/step - loss: 1.0377 - accuracy: 0.6182 - val_loss: 0.9415 - val_accuracy: 0.6985
E

In [45]:
model_7 = Sequential()
model_7.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_7.add(Bidirectional(LSTM(256, return_sequences = True, dropout = 0.2)))
model_7.add(Bidirectional(LSTM(256, return_sequences = True, kernel_regularizer = keras.regularizers.l1())))
model_7.add(Bidirectional(LSTM(128, return_sequences = True, dropout = 0.2)))
model_7.add(Conv1D(256, 3, activation='tanh', padding = 'same'))
model_7.add(Flatten())

model_7.add(Dense(10, activation = 'tanh'))

model_7.add(Dense(1,activation = 'sigmoid'))

model_7.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [46]:
model_7.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 1100, 200)         2900800   
                                                                 
 bidirectional_25 (Bidirecti  (None, 1100, 512)        935936    
 onal)                                                           
                                                                 
 bidirectional_26 (Bidirecti  (None, 1100, 512)        1574912   
 onal)                                                           
                                                                 
 bidirectional_27 (Bidirecti  (None, 1100, 256)        656384    
 onal)                                                           
                                                                 
 conv1d_16 (Conv1D)          (None, 1100, 256)         196864    
                                                     

In [47]:
history_7 = model_7.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=64, 
                    verbose=1, 
                    epochs=15,
                    validation_split=0.2)

Epoch 1/15
39/39 [==============================] - 433s 11s/step - loss: 174.9750 - accuracy: 0.5602 - val_loss: 53.6494 - val_accuracy: 0.5640
Epoch 2/15
39/39 [==============================] - 444s 11s/step - loss: 17.5701 - accuracy: 0.5853 - val_loss: 4.5701 - val_accuracy: 0.5640
Epoch 3/15
39/39 [==============================] - 437s 11s/step - loss: 4.2724 - accuracy: 0.5894 - val_loss: 2.4421 - val_accuracy: 0.5640
Epoch 4/15
39/39 [==============================] - 443s 11s/step - loss: 2.0685 - accuracy: 0.5898 - val_loss: 1.9226 - val_accuracy: 0.5640
Epoch 5/15
39/39 [==============================] - 441s 11s/step - loss: 1.8973 - accuracy: 0.5898 - val_loss: 1.8981 - val_accuracy: 0.5640
Epoch 6/15
39/39 [==============================] - 444s 11s/step - loss: 1.8951 - accuracy: 0.5898 - val_loss: 1.9128 - val_accuracy: 0.5640
Epoch 7/15
39/39 [==============================] - 442s 11s/step - loss: 1.8993 - accuracy: 0.5898 - val_loss: 1.9086 - val_accuracy: 0.5640
Ep

In [48]:
model_8 = Sequential()
model_8.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_8.add(Conv1D(256, 5, activation='tanh', padding = 'same'))
model_8.add(Conv1D(128, 3, activation='tanh', padding = 'same'))
model_8.add(Bidirectional(LSTM(256, return_sequences = True, dropout = 0.2)))
model_8.add(Bidirectional(LSTM(256, return_sequences = True, kernel_regularizer = keras.regularizers.L2(1e-4))))
model_8.add(Bidirectional(LSTM(128)))

model_8.add(Dense(1,activation = 'sigmoid'))

model_8.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [49]:
history_8 = model_8.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=64, 
                    verbose=1, 
                    epochs=8,
                    validation_split=0.2)

Epoch 1/8
39/39 [==============================] - 403s 10s/step - loss: 0.7478 - accuracy: 0.6283 - val_loss: 0.7459 - val_accuracy: 0.6451
Epoch 2/8
39/39 [==============================] - 408s 10s/step - loss: 0.6716 - accuracy: 0.6704 - val_loss: 0.7998 - val_accuracy: 0.5818
Epoch 3/8
39/39 [==============================] - 419s 11s/step - loss: 0.7003 - accuracy: 0.6405 - val_loss: 0.6728 - val_accuracy: 0.6305
Epoch 4/8
39/39 [==============================] - 422s 11s/step - loss: 0.6324 - accuracy: 0.6846 - val_loss: 0.6556 - val_accuracy: 0.6499
Epoch 5/8
39/39 [==============================] - 426s 11s/step - loss: 0.5770 - accuracy: 0.7430 - val_loss: 0.5952 - val_accuracy: 0.7439
Epoch 6/8
39/39 [==============================] - 423s 11s/step - loss: 0.5469 - accuracy: 0.7535 - val_loss: 0.7226 - val_accuracy: 0.5640
Epoch 7/8
39/39 [==============================] - 426s 11s/step - loss: 0.6883 - accuracy: 0.6194 - val_loss: 0.7001 - val_accuracy: 0.6240
Epoch 8/8
39/

In [51]:
model_9 = Sequential()
model_9.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

model_9.add(Conv1D(256, 5, activation='tanh', padding = 'same'))
model_9.add(Conv1D(128, 3, activation='tanh', padding = 'same'))
model_9.add(Bidirectional(LSTM(256, return_sequences = True)))
model_9.add(Bidirectional(LSTM(256, return_sequences = True)))
model_9.add(Bidirectional(LSTM(256, return_sequences = True, dropout = 0.2)))
model_9.add(Bidirectional(LSTM(256, return_sequences = True, dropout = 0.2)))
model_9.add(Bidirectional(LSTM(128, kernel_regularizer = keras.regularizers.L2(1e-4))))

model_9.add(Dense(1,activation = 'sigmoid'))

model_9.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [52]:
model_9.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 1100, 200)         2900800   
                                                                 
 conv1d_19 (Conv1D)          (None, 1100, 256)         256256    
                                                                 
 conv1d_20 (Conv1D)          (None, 1100, 128)         98432     
                                                                 
 bidirectional_31 (Bidirecti  (None, 1100, 512)        788480    
 onal)                                                           
                                                                 
 bidirectional_32 (Bidirecti  (None, 1100, 512)        1574912   
 onal)                                                           
                                                                 
 bidirectional_33 (Bidirecti  (None, 1100, 512)      

In [54]:
history_9 = model_9.fit(pad_seq, 
                    y_filtered_converted,
                    batch_size=128, 
                    verbose=1, 
                    epochs=8,
                    validation_split=0.2)

Epoch 1/8
 6/20 [========>.....................] - ETA: 13:09 - loss: 0.8206 - accuracy: 0.5807

KeyboardInterrupt: 